<a href="https://colab.research.google.com/github/ben900926/Plant-seedling-classification/blob/main/AI_final_project_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import torch
import torchvision
import torch.nn as nn
from glob import glob
from torchvision import models
import torch.utils.data as data 
from torchvision import transforms
from torchvision.datasets.folder import DatasetFolder
from PIL import Image
from tqdm import tqdm
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
root = '/content/drive/MyDrive/AI_FinalProject/plant-seedlings-classification/train'

val_size = 0.2
batch_size = 64
num_workers = 8
num_epochs = 40

Mounted at /content/drive


In [ ]:
# VGG Baseline
class VGG(nn.Module):
    def __init__(self, num_classes):
        super(VGG, self).__init__()
        self.model = models.vgg11(pretrained=True)
        self.model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)

    def forward(self, x):
        return self.model(x)
    
# ResNet Baseline, Resnet18, Resnet34
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.model = models.resnet34(pretrained=True)
        self.model.fc = nn.Linear(in_features=512, out_features=num_classes)

        for module in ['conv1', 'bn1', 'layer1']:
            for param in getattr(self.model, module).parameters():
                param.requires_grad = False
    def forward(self, x):
        return self.model(x)
             
#Densenet Baseline, Densenet 121
class Densenet(nn.Module): 
    def __init__(self, num_classes):
        super(Densenet, self).__init__()
        self.model = models.densenet121(pretrained=True)
        self.model.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        return self.model(x)
    
model = ResNet(num_classes=12)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
test_paths = glob('/content/drive/MyDrive/AI_FinalProject/plant-seedlings-classification/test/*.png')

In [ ]:
train_paths = []
val_paths = []
train_labels = []
val_labels = []

In [ ]:
class_paths = glob(root + '/*')

classes = [path.split("/")[-1] for path in class_paths]

for i, cpath in enumerate(class_paths):
    paths = glob(cpath + '/*.png')
    train_split = int(len(paths) * 0.8)
    
    train_paths.extend(paths[:train_split])
    train_labels.extend([i] * train_split)
    
    val_paths.extend(paths[train_split:])
    val_labels.extend([i] *(len(paths) - train_split))

In [ ]:
class Dataset(data.Dataset):

    def __init__(self, img_paths, img_labels=None, transform=None):
        self.transform = transform
        self.img_paths = img_paths
        self.img_labels = img_labels

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        if self.img_labels is not None:
            label = self.img_labels[idx]
            return img, label
        else:
            return img, idx

    def __len__(self):
        return len(self.img_paths)
    
train_transforms = transforms.Compose([transforms.Resize((256, 256)),
                                       transforms.RandomRotation(degrees=(-8, 8)),
                                       transforms.RandomCrop((224, 224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ColorJitter(brightness=0.2, contrast=0.2,
                                                              saturation=0.2, hue=0.2),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.CenterCrop((224, 224)),
                                     transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])])

train_dataset = Dataset(train_paths, train_labels, transform=train_transforms)
val_dataset = Dataset(val_paths, val_labels, transform=val_transforms)
test_dataset = Dataset(test_paths,transform=val_transforms)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         shuffle=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
criterion = CrossEntropyLoss()
model = model.cuda()
optimizer = Adam(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0.

    steps = len(train_loader.dataset) // batch_size + 1
    with tqdm(total=steps) as progress_bar:
        
        for i, (x, y) in enumerate(train_loader):
            x, y = x.cuda(), y.cuda()

            optimizer.zero_grad() 
            
            y_pred = model(x)
            
            loss = criterion(y_pred, y)
            
            loss.backward()
            
            optimizer.step()
            
            progress_bar.set_postfix(loss=loss.item())
            progress_bar.update(1)
            
            correct += torch.sum(torch.argmax(y_pred, dim=-1) == y)
            
        train_acc = float(correct.item()) / float(len(train_loader.dataset))
        print("Epoch %d: train correct: %.4f" % (epoch, train_acc))
        
    
    steps = len(val_loader.dataset) // batch_size + 1
    correct = 0.
    model.eval()

    with tqdm(total=steps) as progress_bar:
        for i, (x, y) in enumerate(val_loader):
            x, y = x.cuda(), y.cuda()
            
            y_pred = model(x)
            
            loss = criterion(y_pred, y)
            
            progress_bar.set_postfix(loss=loss.item())
            progress_bar.update(1)
            
            correct += torch.sum(torch.argmax(y_pred, dim=-1) == y)
        
        val_acc = float(correct.item()) / float(len(val_loader.dataset))
        print("Epoch %d: val correct: %.4f" % (epoch, val_acc))

  0%|          | 0/48 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 48/48 [01:57<00:00,  2.45s/it, loss=0.448]


Epoch 0: train correct: 0.6636


100%|██████████| 12/12 [00:35<00:00,  2.96s/it, loss=0.0536]


Epoch 0: val correct: 0.8458


100%|██████████| 48/48 [01:01<00:00,  1.28s/it, loss=0.208]


...

Epoch 38: val correct: 0.9412


100%|██████████| 48/48 [00:59<00:00,  1.23s/it, loss=0.0132]


Epoch 39: train correct: 0.9947


100%|██████████| 12/12 [00:11<00:00,  1.06it/s, loss=0.0178]

Epoch 39: val correct: 0.9425


In [ ]:
test_label_indices = []
test_img_indices = []

for i, (x, img_idx) in enumerate(test_loader):
    x = x.cuda()
    y_pred = model(x)
    test_label_indices.extend(list(torch.argmax(y_pred, dim=-1).cpu().numpy()))
    test_img_indices.extend(list(img_idx.cpu().numpy()))
    
test_names = [test_paths[idx] for idx in test_img_indices]
test_names = [name.split("/")[-1] for name in test_names]
test_labels = [classes[idx] for idx in test_label_indices]

out_df = pd.DataFrame({'file': test_names,'species' : test_labels})
out_df.to_csv('ResNet-34.csv', index=False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
